In [1]:
import pysam
import pandas as pd
#import pyBigWig
import pybedtools

In [4]:
vcf_file_path = "/home/pdutta/Data/GDC/RAMANA/data/VCF/274f9dc8-eac5-4a19-8e22-f3af2201f50f.rna_seq.transcriptome.gdc_realn.vcf"
reference_file_path = "/home/pdutta/Rekha_LabWork/Collaborator_work/RNA-seq/RNAseq_CommonFiles/STAR/Genome/GRCh38.primary_assembly.genome.fa"

In [5]:
vcf_file = pysam.VariantFile(vcf_file_path)
reference_fasta = pysam.FastaFile(reference_file_path)

In [6]:
for rec in vcf_file.fetch():
    # You can access all data about the variant like this:
    print('Chromosome:', rec.chrom)
    print('Position:', rec.pos)
    print('Reference allele:', rec.ref)
    print('Alternative alleles:', rec.alts)

In [14]:
context = 50
data = []

for rec in vcf_file.fetch():
    # Get the sequence context around the variant
    seq_context = reference_fasta.fetch("chr"+rec.chrom, rec.pos - 1 - context, rec.pos + context)

    # Replace the reference allele with the alternative allele
    alt_seq_context = seq_context[:context] + str(rec.alts[0]) + seq_context[context + len(rec.ref):]

    print('Reference Allele and alterna', rec.ref , rec.alts)
    print('Position:', rec.pos)
    print("ID", rec.id)
    data.append([rec.chrom, rec.pos, rec.id, seq_context, alt_seq_context])
    print(seq_context)
    print(alt_seq_context)

Reference Allele and alterna G ('A',)
Position: 44905879
ID rs373985746
TCCCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTC
TCCCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGAGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTC
Reference Allele and alterna G ('T',)
Position: 44905881
ID None
CCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTCTC
CCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGTCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTCTC
Reference Allele and alterna T ('C',)
Position: 44905886
ID None
GAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTCTCAGGAG
GAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGGCCTCCAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTCTCAGGAG
Reference Allele and alterna C ('G',)
Position: 44905910
ID rs440446
GGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTCTCAGGAGAGCTACTCGGGGTCGGGCTTGGGG
GGCACGGGGATGAGC

In [15]:
# Create a dataframe from your data
df_APOE = pd.DataFrame(data, columns=["CHROMOSOME", "POS",'ID', 'REFERENCE_SEQUENCE', 'ALTERNATIVE_SEQUENCE'])
df_APOE

,CHROMOSOME,POS,ID,REFERENCE_SEQUENCE,ALTERNATIVE_SEQUENCE
0,19,44905879,rs373985746,TCCCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGC...,TCCCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGC...
1,19,44905881,None,CCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTC...,CCCAGGAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTC...
2,19,44905886,None,GAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGG...,GAGCCGGTGAGAAGCGCAGTCGGGGGCACGGGGATGAGCTCAGGGG...
3,19,44905910,rs440446,GGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTG...,GGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTG...
4,19,44905923,None,GCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCC...,GCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGGGAACCCCTGGCC...
5,19,44906639,None,TCTCCTTCCCCAGACTGGCCAATCACAGGCAGGAAGATGAAGGTTC...,TCTCCTTCCCCAGACTGGCCAATCACAGGCAGGAAGATGAAGGTTC...
6,19,44906646,None,CCCCAGACTGGCCAATCACAGGCAGGAAGATGAAGGTTCTGTGGGC...,CCCCAGACTGGCCAATCACAGGCAGGAAGATGAAGGTTCTGTGGGC...
7,19,44907785,rs111833428,CCCACCTTGAACTTGTTCCACACAGGATGCCAGGCCAAGGTGGAGC...,CCCACCTTGAACTTGTTCCACACAGGATGCCAGGCCAAGGTGGAGC...
8,19,44907788,None,ACCTTGAACTTGTTCCACACAGGATGCCAGGCCAAGGTGGAGCAAG...,ACCTTGAACTTGTTCCACACAGGATGCCAGGCCAAGGTGGAGCAAG...
9,19,44907934,None,CTGGGTGCAGACACTGTCTGAGCAGGTGCAGGAGGAGCTGCTCAGC...,CTGGGTGCAGACACTGTCTGAGCAGGTGCAGGAGGAGCTGCTCAGC...


In [30]:
data= []
for rec in vcf.fetch():
    for sample in rec.samples:
        sample_data = rec.samples[sample]
        data.append([rec.chrom, rec.pos, rec.id, rec.ref, ','.join(rec.alts), rec.qual,
                     ';'.join(rec.filter.keys()), dict(rec.info), sample, sample_data['GT']])

# Create a pandas DataFrame from the data
df = pd.DataFrame(data, columns=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'SAMPLE', 'GT'])


In [43]:
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,SAMPLE,GT
0,19,44905879,rs373985746,G,A,1767.01001,PASS,"{'AC': (1,), 'AF': (2.418000076431781e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
1,19,44905879,rs373985746,G,A,1767.01001,PASS,"{'AC': (1,), 'AF': (2.418000076431781e-05,), '...",A-ACT-AC000008-BL-UPN-8307,"(0, 0)"
2,19,44905879,rs373985746,G,A,1767.01001,PASS,"{'AC': (1,), 'AF': (2.418000076431781e-05,), '...",A-ACT-AC000010-BL-UWA-15286,"(0, 0)"
3,19,44905879,rs373985746,G,A,1767.01001,PASS,"{'AC': (1,), 'AF': (2.418000076431781e-05,), '...",A-ACT-AC000014-BL-UPN-23967,"(0, 0)"
4,19,44905879,rs373985746,G,A,1767.01001,PASS,"{'AC': (1,), 'AF': (2.418000076431781e-05,), '...",A-ACT-AC000016-BL-UPN-7273,"(0, 0)"
...,...,...,...,...,...,...,...,...,...,...
94565,19,44907934,None,A,G,804.77002,PASS,"{'AC': (1,), 'AF': (2.370999936829321e-05,), '...",C-RS-52108-BL-ERA-6739002,"(0, 0)"
94566,19,44907934,None,A,G,804.77002,PASS,"{'AC': (1,), 'AF': (2.370999936829321e-05,), '...",C-RS-52109-BL-ERA-1267001,"(0, 0)"
94567,19,44907934,None,A,G,804.77002,PASS,"{'AC': (1,), 'AF': (2.370999936829321e-05,), '...",C-RS-52110-BL-ERA-2326001,"(0, 0)"
94568,19,44907934,None,A,G,804.77002,PASS,"{'AC': (1,), 'AF': (2.370999936829321e-05,), '...",C-RS-52111-BL-ERA-3023001,"(0, 0)"


In [44]:
df[df["SAMPLE"]=="A-ACT-AC000007-BL-UPN-6888"]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,SAMPLE,GT
0,19,44905879,rs373985746,G,A,1.767010e+03,PASS,"{'AC': (1,), 'AF': (2.418000076431781e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
9457,19,44905881,None,G,T,6.200700e+02,PASS,"{'AC': (1,), 'AF': (7.257000106619671e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
18914,19,44905886,None,T,C,9.460500e+02,PASS,"{'AC': (2,), 'AF': (4.8379999498138204e-05,), ...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
28371,19,44905910,rs440446,C,G,2.124650e+07,PASS,"{'AC': (12205,), 'AF': (0.7009999752044678,), ...",A-ACT-AC000007-BL-UPN-6888,"(1, 1)"
37828,19,44905923,None,G,A,1.645320e+04,PASS,"{'AC': (27,), 'AF': (0.0008941999985836446,), ...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
47285,19,44906639,None,G,A,7.705500e+02,PASS,"{'AC': (2,), 'AF': (4.741999873658642e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
56742,19,44906646,None,T,C,1.157700e+02,PASS,"{'AC': (1,), 'AF': (2.370999936829321e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
66199,19,44907785,rs111833428,G,A,2.345130e+04,PASS,"{'AC': (12,), 'AF': (0.0004980000085197389,), ...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
75656,19,44907788,None,G,A,2.777900e+02,PASS,"{'AC': (1,), 'AF': (4.742999954032712e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"
85113,19,44907934,None,A,G,8.047700e+02,PASS,"{'AC': (1,), 'AF': (2.370999936829321e-05,), '...",A-ACT-AC000007-BL-UPN-6888,"(0, 0)"


In [34]:
fasta.fetch('chr19', 10, 20)

'NNNNNNNNNN'

Chromosome: G ('A',)
Position: 44905879
ATGAGCTCAGGGGCCTCTAGA
ATGAGCTCAGAGGCCTCTAGA


Chromosome: G ('T',)
Position: 44905881
GAGCTCAGGGGCCTCTAGAAA
GAGCTCAGGGTCCTCTAGAAA


Chromosome: T ('C',)
Position: 44905886
CAGGGGCCTCTAGAAAGAGCT
CAGGGGCCTCCAGAAAGAGCT


Chromosome: C ('G',)
Position: 44905910
ACCCTGGGAACCCCTGGCCTC
ACCCTGGGAAGCCCTGGCCTC


Chromosome: G ('A',)
Position: 44905923
CTGGCCTCCAGGTAGTCTCAG
CTGGCCTCCAAGTAGTCTCAG


Chromosome: G ('A',)
Position: 44906639
AGGTTCTGTGGGCTGCGTTGC
AGGTTCTGTGAGCTGCGTTGC


Chromosome: T ('C',)
Position: 44906646
GTGGGCTGCGTTGCTGGTCAC
GTGGGCTGCGCTGCTGGTCAC


Chromosome: G ('A',)
Position: 44907785
TGGAGCAAGCGGTGGAGACAG
TGGAGCAAGCAGTGGAGACAG


Chromosome: G ('A',)
Position: 44907788
AGCAAGCGGTGGAGACAGAGC
AGCAAGCGGTAGAGACAGAGC


Chromosome: A ('G',)
Position: 44907934
CTCAGCTCCCAGGTCACCCAG
CTCAGCTCCCGGGTCACCCAG


In [49]:
def get_flanking_sequence(fasta, chrom, position, flanking_bp):
    start = max(0, position - flanking_bp - 1)
    end = position + flanking_bp
    seq = fasta.fetch(chrom, start, end)
    return seq

In [52]:
# replace with your files
vcf_filename = file_path
reference_fasta_path = "/data/projects/Resources/HumanReferenceGenome/chr19.fa"

In [53]:
vcf = pysam.VariantFile(vcf_filename)
fasta = pysam.FastaFile(reference_fasta_path)
flanking_bp = 20

for rec in vcf:
    ref_seq = rec.ref
    alt_seq = rec.alts[0] # assuming there is always at least one alternate allele
    seq = get_flanking_sequence(fasta, "chr"+rec.chrom, rec.pos, flanking_bp)

    print(f'Position: {rec.pos}')
    print(f'Reference allele: {ref_seq}')
    print(f'Alternate allele: {alt_seq}')
    print(f'Flanking sequence: {seq}')
    input()

Position: 44905879
Reference allele: G
Alternate allele: A
Flanking sequence: GGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGG


Position: 44905881
Reference allele: G
Alternate allele: T
Flanking sequence: GCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGAC


Position: 44905886
Reference allele: T
Alternate allele: C
Flanking sequence: GGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGGACCCTGG


Position: 44905910
Reference allele: C
Alternate allele: G
Flanking sequence: AAGAGCTGGGACCCTGGGAACCCCTGGCCTCCAGGTAGTCT


Position: 44905923
Reference allele: G
Alternate allele: A
Flanking sequence: CTGGGAACCCCTGGCCTCCAGGTAGTCTCAGGAGAGCTACT


Position: 44906639
Reference allele: G
Alternate allele: A
Flanking sequence: AGGAAGATGAAGGTTCTGTGGGCTGCGTTGCTGGTCACATT


Position: 44906646
Reference allele: T
Alternate allele: C
Flanking sequence: TGAAGGTTCTGTGGGCTGCGTTGCTGGTCACATTCCTGGCA


Position: 44907785
Reference allele: G
Alternate allele: A
Flanking sequence: CAGGCCAAGGTGGAGCAAGCGGTGGAGACAGAGCCGGAGCC


Position: 44907788
Reference allele: G
Alternate allele: A
Flanking sequence: GCCAAGGTGGAGCAAGCGGTGGAGACAGAGCCGGAGCCCGA


Position: 44907934
Reference allele: A
Alternate allele: G
Flanking sequence: GGAGGAGCTGCTCAGCTCCCAGGTCACCCAGGAACTGAGGT


In [57]:
vcf = pysam.VariantFile(vcf_filename)
fasta = pysam.FastaFile(reference_fasta_path)
flanking_bp = 20

for rec in vcf:
    for sample in rec.samples:
        genotype = rec.samples[sample]['GT']
        if genotype is None:  # genotype information not available for this sample
            continue
        alleles = [rec.ref] + list(rec.alts) if rec.alts else [rec.ref]
        sample_alleles = [alleles[gt] for gt in genotype if gt is not None]

        print(f'Sample: {sample}')
        print(f'Genotype: {sample_alleles}')

        for allele in sample_alleles:
            seq = get_flanking_sequence(fasta, "chr"rec.chrom, rec.pos, flanking_bp)
            print(f'Position: {rec.pos}')
            print(f'Allele: {allele}')
            print(f'Flanking sequence: {seq}')
            input()

Sample: A-ACT-AC000007-BL-UPN-6888
Genotype: ['G', 'G']
Position: 44905879
Allele: G
Flanking sequence: GGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGG


Position: 44905879
Allele: G
Flanking sequence: GGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGG


Sample: A-ACT-AC000008-BL-UPN-8307
Genotype: ['G', 'G']
Position: 44905879
Allele: G
Flanking sequence: GGGCACGGGGATGAGCTCAGGGGCCTCTAGAAAGAGCTGGG


KeyboardInterrupt: Interrupted by user

In [7]:
df_test.columns.to_list()

['CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'A-ACT-AC000007-BL-UPN-6888',
 'A-ACT-AC000008-BL-UPN-8307',
 'A-ACT-AC000010-BL-UWA-15286',
 'A-ACT-AC000014-BL-UPN-23967',
 'A-ACT-AC000016-BL-UPN-7273',
 'A-ACT-AC000020-BL-UWA-13783',
 'A-ACT-AC000022-BL-UWA-15061',
 'A-ACT-AC000023-BL-UPN-5880',
 'A-ACT-AC000025-BL-UPN-10505',
 'A-ACT-AC000027-BL-UPN-23154',
 'A-ACT-AC000032-BL-UPN-23447',
 'A-ACT-AC000034-BL-UPN-15865',
 'A-ACT-AC000037-BL-UPN-7487',
 'A-ACT-AC000038-BL-UWA-32410',
 'A-ACT-AC000039-BL-UWA-14822',
 'A-ACT-AC000040-BL-UPN-7990',
 'A-ACT-AC000041-BL-UWA-14269',
 'A-ACT-AC000043-BL-UPN-11291',
 'A-ACT-AC000044-BL-UPN-11405',
 'A-ACT-AC000045-BL-UPN-26144',
 'A-ACT-AC000046-BL-UPN-14126',
 'A-ACT-AC000047-BL-UPN-10604',
 'A-ACT-AC000055-BL-UPN-14910',
 'A-ACT-AC000056-BL-UPN-15169',
 'A-ACT-AC000057-BL-UPN-14863',
 'A-ACT-AC000058-BL-UPN-14011',
 'A-ACT-AC000064-BL-UWA-14740',
 'A-ACT-AC000067-BL-UPN-11551',
 'A-ACT-AC000071-BL-UWA-14864